In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from pybit import inverse_perpetual  # <-- import HTTP & WSS for inverse perp
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import pandas as pd
import time
import ta 
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [2]:
load_dotenv()

True

In [3]:
#Loading my Bybit's API keys from the dotenv file
api_key_pw = os.getenv('api_key_bot_IP')
api_secret_pw = os.getenv('api_secret_bot_IP')

In [4]:
#Stablishing Connection with the API (SPOT)
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)

#Creating a Dataframe of ETH real price (1m)
frame = pd.DataFrame(session_auth.query_kline(symbol="ETHUSDT", interval="1m")["result"])
frame

,0,1,2,3,4,5,6,7,8,9,10
0,1671333480000,1185.84,1185.9,1185.65,1185.9,0.59251,1671333540000,702.657049,2,0,0
1,1671333540000,1185.9,1186.02,1185.65,1185.9,0.64825,1671333600000,768.6645366,6,0,0
2,1671333600000,1185.9,1185.9,1185.59,1185.84,7.30481,1671333660000,8660.9098411,7,0,0
3,1671333660000,1185.84,1185.84,1185.57,1185.57,7.22821,1671333720000,8569.7239297,6,0,0
4,1671333720000,1185.57,1185.66,1185.46,1185.66,0.62664,1671333780000,742.945194,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1671393180000,1181.95,1182.15,1181.68,1182.15,9.56864,1671393240000,11309.3652716,29,0,0
996,1671393240000,1182.15,1183.15,1182.15,1182.96,76.67807,1671393300000,90706.4332765,107,0,0
997,1671393300000,1182.96,1183.22,1182.7,1182.76,4.70682,1671393360000,5567.3477131,27,0,0
998,1671393360000,1182.76,1183.43,1182.61,1182.95,38.0649,1671393420000,45033.734415,80,0,0


In [5]:
#This function gets Real BTC Price Data and creates a smooth dataframe that refreshes every 15 minutes
def get5minutedata():
    frame = pd.DataFrame(session_auth.query_kline(symbol="ETHUSDT", interval="5m")["result"])
    frame = frame.iloc[:,: 6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [6]:
df = get5minutedata()
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-12-15 08:40:00,1288.00,1288.24,1286.84,1287.76,105.67475
2022-12-15 08:45:00,1287.76,1288.02,1287.12,1287.25,58.29746
2022-12-15 08:50:00,1287.25,1287.25,1284.16,1285.58,301.45021
2022-12-15 08:55:00,1285.58,1285.62,1283.67,1283.67,90.76506
2022-12-15 09:00:00,1283.67,1285.26,1282.48,1285.07,260.92069
...,...,...,...,...,...
2022-12-18 19:35:00,1181.86,1182.31,1180.99,1181.38,61.75310
2022-12-18 19:40:00,1181.38,1183.48,1181.38,1183.48,53.91226
2022-12-18 19:45:00,1183.48,1183.49,1181.09,1181.39,84.54712


In [7]:
#Function to apply some technical indicators from the ta library
def apply_technicals(df):
    df["K"] = ta.momentum.stochrsi(df.Close, window= 14)
    #df["D"] = df["K"].rolling(3).mean()
    df["RSI"] = ta.momentum.rsi(df.Close, window = 14)
    df.dropna(inplace=True)

In [8]:
apply_technicals(df)

In [9]:
class Signals:
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
    
    #Checking if we have a trigger in the last n time steps
    def get_trigger(self):
        df_2 = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df["RSI"].shift(i) > 75)
            df_2 = df_2.append(mask, ignore_index = True)
        return df_2.sum(axis= 0)
    
    # Is the trigger fulfilled and are all buying conditions fulfilled?
    def decide(self):
         self.df["trigger"] = np.where(self.get_trigger(), 1, 0)
         self.df["Sell"]= np.where((self.df.trigger), 1, 0)

In [10]:
inst = Signals(df, 1)

inst.decide()

In [11]:
df[df.Sell==1]

,Open,High,Low,Close,Volume,K,RSI,trigger,Sell
Time,,,,,,,,,
2022-12-16 07:10:00,1275.11,1278.17,1274.85,1277.79,271.72118,1.000000,75.148961,1,1
2022-12-16 07:15:00,1277.79,1279.95,1276.74,1277.43,358.43914,0.944754,73.180660,1,1
2022-12-17 23:05:00,1185.38,1189.97,1184.95,1186.60,378.23390,1.000000,75.655661,1,1
2022-12-17 23:10:00,1186.60,1186.79,1183.61,1184.43,175.99512,0.649459,64.603244,1,1
2022-12-18 04:10:00,1188.85,1193.92,1188.85,1193.23,477.25135,1.000000,84.405178,1,1
2022-12-18 04:15:00,1193.23,1194.09,1191.22,1191.55,152.26870,0.590217,72.203906,1,1
2022-12-18 04:20:00,1191.55,1196.48,1191.10,1194.00,743.34396,0.762942,77.346795,1,1
2022-12-18 04:25:00,1194.00,1195.76,1194.00,1194.86,242.09583,0.812677,78.827640,1,1
2022-12-18 04:30:00,1194.86,1196.40,1193.34,1193.78,236.56499,0.597633,72.424737,1,1


In [12]:
#The mail addresses and password
sender_address = 'pythontradingbot11@gmail.com'
sender_pass = os.getenv('mail_key')
receiver_address = 'carlos.mfresco@gmail.com'
#Setup the MIME
message = MIMEMultipart() 
message_SL = MIMEMultipart()
message_TP = MIMEMultipart()
message_RSI = MIMEMultipart()
message_Others = MIMEMultipart()

In [13]:
def strategy_short(qty, open_position = False):
    df= get5minutedata()
    apply_technicals(df)
    inst = Signals(df, 1)
    inst.decide()
    print(f'Current Close is '+str(df.Close.iloc[-1]))
    print(f'Current RSI is ' + str(df.RSI.iloc[-1]))
    print("-----------------------------------------")

    if df.Sell.iloc[-1]:
        mail_content = "ETH Open Short"
        message.attach(MIMEText(mail_content, 'plain'))
        
        # Create SMTP session for sending the mail
        session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
        session_mail.starttls()  # enable security

        # login with mail_id and password
        session_mail.login(sender_address, sender_pass)
        text = message.as_string()
        session_mail.sendmail(sender_address, receiver_address, text)
        session_mail.quit()

        from pybit import usdt_perpetual
        session = usdt_perpetual.HTTP(
        endpoint='https://api.bybit.com',
        api_key = api_key_pw,
        api_secret= api_secret_pw)

        buyprice = round(df.Close.iloc[-1],2)

        print("-----------------------------------------")

        print(f"Buyprice: {buyprice}")

        print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

        order = session.place_active_order(symbol="ETHUSDT",
                                                side="Sell",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=False,
                                                close_on_trigger=False,
                                                take_profit = round(buyprice * 0.93,2),
                                                stop_loss = round(buyprice * 1.03,2))
        print(order)

        eth_order_id = str(order['result']['order_id'])
        print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
        print(f"Order id: {eth_order_id}") 
        print("-------------------------------------------------")

        open_position = True
    while open_position:
        time.sleep(30)
        from pybit import spot
        session_auth = spot.HTTP(
            endpoint='https://api.bybit.com',
            api_key = api_key_pw,
            api_secret= api_secret_pw)
            
        df = get5minutedata()
        apply_technicals(df)
        print(f"Buyprice: {buyprice}" + '                Close: ' + str(df.Close.iloc[-1]))
        print(f'Target: ' + str(round(buyprice * 0.93, 2)) + "                 Stop: " + str(round(buyprice * 1.03, 2)))
        print(f'RSI Target: 25' + '                RSI: ' + str(df.RSI.iloc[-1]))
        print("---------------------------------------------------")

        if df.Close[-1] > buyprice* 1.03:
            print("Closed Position")
            open_position = False

            mail_content_SL = "ETH Short SL"
            message_SL.attach(MIMEText(mail_content_SL, 'plain'))

            # Create SMTP session for sending the mail
            session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
            session_mail.starttls()  # enable security

            # login with mail_id and password
            session_mail.login(sender_address, sender_pass)
            text = message_SL.as_string()
            session_mail.sendmail(sender_address, receiver_address, text)
            session_mail.quit()
            break

        elif df.Close[-1] < buyprice * 0.93: 
            print("Closed Position")
            open_position = False

            mail_content_TP = "ETH Short TP"
            message_TP.attach(MIMEText(mail_content_TP, 'plain'))

            # Create SMTP session for sending the mail
            session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
            session_mail.starttls()  # enable security

            # login with mail_id and password
            session_mail.login(sender_address, sender_pass)
            text = message_TP.as_string()
            session_mail.sendmail(sender_address, receiver_address, text)
            session_mail.quit()

        elif df.RSI[-1] < 25:
            session = usdt_perpetual.HTTP(
            endpoint='https://api.bybit.com',
            api_key = api_key_pw,
            api_secret= api_secret_pw)

            try: 
                print(session.place_active_order(symbol="ETHUSDT",
                                                side="Buy",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=True,
                                                close_on_trigger=False))            
            
                print("Closed position")
                open_position = False
                mail_content_RSI = "ETH Short Closed - RSI < 25"
                message_RSI.attach(MIMEText(mail_content_RSI, 'plain'))

                # Create SMTP session for sending the mail
                session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
                session_mail.starttls()  # enable security

                # login with mail_id and password
                session_mail.login(sender_address, sender_pass)
                text = message_RSI.as_string()
                session_mail.sendmail(sender_address, receiver_address, text)
                session_mail.quit()
                break

            except: 
                print("Position already closed")
                
                mail_content_Others = "Position Closed"
                message_Others.attach(MIMEText(mail_content_Others, 'plain'))

                # Create SMTP session for sending the mail
                session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
                session_mail.starttls()  # enable security

                # login with mail_id and password
                session_mail.login(sender_address, sender_pass)
                text = message_Others.as_string()
                session_mail.sendmail(sender_address, receiver_address, text)
                session_mail.quit()
                break

In [32]:
while True: 
    strategy_short(0.5)
    time.sleep(60)

Current Close is 1182.9
Current RSI is 55.818625490065614
-----------------------------------------


: 

In [ ]:
#ta.volume.VolumeWeightedAveragePrice(high: pandas.core.series.Series, low: pandas.core.series.Series, close: pandas.core.series.Series, volume: pandas.core.series.Series, window: int = 14, fillna: bool = False)